In [14]:
## Import Utilities
import os
from google.colab import drive

## Import Core files
import numpy as np
import librosa
import tensorflow as tf
import tensorflow_hub as hub
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [ ]:
# Upload genres.tar.gz to GOOGLE DRIVE and complete the Authorization flow
drive.mount('/content/drive')
#Execute shell command to UNtar and UNzip
!tar -zxvf '/content/drive/MyDrive/genres.tar.gz'

# New section

In [ ]:
# YAMNet embedding extraction
yamnet_model = hub.load('https://tfhub.dev/google/yamnet/1')

def extract_yamnet_embedding(file_path):
    try:
        audio, sr = librosa.load(file_path, sr=16000, mono=True)
        if len(audio) == 0:
            print("Empty audio:", file_path)
            return None

        waveform = tf.convert_to_tensor(audio, dtype=tf.float32)
        scores, embeddings, spectrogram = yamnet_model(waveform)
        mean_embedding = tf.reduce_mean(embeddings, axis=0)
        return mean_embedding.numpy()
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None

In [ ]:
# Dataset Preparation using YAMNet embeddings
def load_yamnet_data(dataset_path='/content/genres'):
    features, labels = [], []
    genres = [g for g in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, g))]

    print(f"Genres found: {genres}")

    for genre in genres:
        genre_path = os.path.join(dataset_path, genre)
        # Ensure files are listed correctly, potentially looking into subdirectories if needed
        files = [f for f in os.listdir(genre_path) if f.endswith('.au')]
        print(f"{genre}: {len(files)} files")

        for file in files:
            file_path = os.path.join(genre_path, file)
            embedding = extract_yamnet_embedding(file_path)
            if embedding is not None:
                features.append(embedding)
                labels.append(genre)
            else:
                print(f"Skipped file due to empty embedding: {file_path}")

    print(f"\n Extracted {len(features)} valid audio embeddings.")
    return np.array(features), np.array(labels)

In [ ]:
# [VRFY] Folder Lists
print("Top-level folders:", os.listdir('.'))
if os.path.exists('genres'):
    print("Files/Dir Inside genres folder:", os.listdir('genres'))

In [20]:
# Label Encoding and TRAIN TEST split
def prepare_data(X, y):
    encoder = LabelEncoder()
    y_encoded = encoder.fit_transform(y)
    y_categorical = to_categorical(y_encoded)
    return train_test_split(X, y_categorical, test_size=0.2, random_state=42), encoder


In [22]:
# Model Building with YAMNet embeddings
def build_model(input_shape, num_classes):
    model = Sequential()
    model.add(Dense(256, activation='relu', input_shape=(input_shape,)))
    model.add(Dropout(0.4))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(num_classes, activation='softmax'))
#Model COMPILE
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
# MAIN LOOP
if __name__ == "__main__":
    print("Extracting embeddings from audio...")
    X, y = load_yamnet_data()
    print(f"Loaded {len(X)} samples with shape {X[0].shape}")
    (X_train, X_val, y_train, y_val), label_encoder = prepare_data(X, y)
    model = build_model(X.shape[1], y_train.shape[1])
    model.summary()

In [ ]:
    # Callbacks: Early stopping and best model checkpointing
    early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    checkpoint = ModelCheckpoint("best_model.h5", monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)

    print("Training model...")
    history = model.fit(
        X_train, y_train,
        epochs=100,
        batch_size=32,
        validation_data=(X_val, y_val),
        callbacks=[early_stop, checkpoint]
    )

    # Final evaluation
    val_loss, val_acc = model.evaluate(X_val, y_val)
    print(f"\n=======\n Final Validation Accuracy: {val_acc:.4f}\n=======\n")